# Philadelphia EUI

In [1]:
import pandas as pd
#The data comes from https://www.phila.gov/2024-06-20-city-releases-update-on-building-energy-programs-and-interactive-data-tool/
# --> https://opendataphilly.org/datasets/large-building-energy-benchmarking-data/
# --> 2023 Properties Reported (CSV)
# --> Metadata: https://metadata.phila.gov/#home/datasetdetails/5543865220583086178c4eb2/representationdetails/673f8b8123af7d02aff55188/?view_287_page=2

eui_philadelphia_path = "https://drive.google.com/uc?id=1RIbveqcoHHiuA3XaZDKKZl4S1GzNLWWO"
eui_philadelphia_df = pd.read_csv(eui_philadelphia_path)
eui_philadelphia_df = eui_philadelphia_df.rename(columns={'y_lat': 'latitude', 'x_lon': 'longitude'})




In [2]:
property_type_classification = {
    # Residential
    "Multifamily Housing": "Residential",
    "Residence Hall/Dormitory": "Residential",
    "Residential Care Facility": "Residential",
    "Other - Lodging/Residential": "Residential",

    # Non-Residential
    "Office": "Non-Residential",
    "Distribution Center": "Non-Residential",
    "Non-Refrigerated Warehouse": "Non-Residential",
    "K-12 School": "Non-Residential",
    "Self-Storage Facility": "Non-Residential",
    "Retail Store": "Non-Residential",
    "Hotel": "Non-Residential",
    "Manufacturing/Industrial Plant": "Non-Residential",
    "Strip Mall": "Non-Residential",
    "Supermarket/Grocery Store": "Non-Residential",
    "Medical Office": "Non-Residential",
    "Senior Living Community": "Non-Residential",
    "Wholesale Club/Supercenter": "Non-Residential",
    "Parking": "Non-Residential",
    "College/University": "Non-Residential",
    "Refrigerated Warehouse": "Non-Residential",
    "Mixed Use Property": "Non-Residential",
    "Laboratory": "Non-Residential",
    "Hospital (General Medical & Surgical)": "Non-Residential",
    "Movie Theater": "Non-Residential",
    "Courthouse": "Non-Residential",
    "Vehicle Dealership": "Non-Residential",
    "Other - Public Services": "Non-Residential",
    "Lifestyle Center": "Non-Residential",
    "Worship Facility": "Non-Residential",
    "Enclosed Mall": "Non-Residential",
    "Other - Mall": "Non-Residential",
    "Data Center": "Non-Residential",
    "Fitness Center/Health Club/Gym": "Non-Residential",
    "Other - Technology/Science": "Non-Residential",
    "Prison/Incarceration": "Non-Residential",
    "Police Station": "Non-Residential",
    "Other - Education": "Non-Residential",
    "Repair Services (Vehicle, Shoe, Locksmith, etc.)": "Non-Residential",
    "Transportation Terminal/Station": "Non-Residential",
    "Library": "Non-Residential",
    "Other - Specialty Hospital": "Non-Residential",
    "Urgent Care/Clinic/Other Outpatient": "Non-Residential",
    "Other - Entertainment/Public Assembly": "Non-Residential",
    "Financial Office": "Non-Residential",
    "Other - Recreation": "Non-Residential",
    "Other - Services": "Non-Residential",
    "Museum": "Non-Residential",
    "Performing Arts": "Non-Residential",
    "Bank Branch": "Non-Residential",
    "Convention Center": "Non-Residential",
    "Outpatient Rehabilitation/Physical Therapy": "Non-Residential",
    "Casino": "Non-Residential",
    "Adult Education": "Non-Residential",
    "Bowling Alley": "Non-Residential",
    "Social/Meeting Hall": "Non-Residential",
    "Energy/Power Station": "Non-Residential",
    "Food Sales": "Non-Residential",
    "Other - Utility": "Non-Residential",
    "Restaurant": "Non-Residential",
    "Vocational School": "Non-Residential",
    "Bar/Nightclub": "Non-Residential",
    "Indoor Arena": "Non-Residential",
    "Pre-school/Daycare": "Non-Residential",
    "Stadium (Open)": "Non-Residential",
    "Ambulatory Surgical Center": "Non-Residential",
    "Drinking Water Treatment & Distribution": "Non-Residential",
    "Fire Station": "Non-Residential",
    "Food Service": "Non-Residential",
    "Ice/Curling Rink": "Non-Residential",
    "Other - Restaurant/Bar": "Non-Residential",
    "Other - Stadium": "Non-Residential",
    "Other/Specialty Hospital": "Non-Residential",
    "Vehicle Repair Services": "Non-Residential",
    "Veterinary Office": "Non-Residential",
    "Fast Food Restaurant": "Non-Residential",
    "Personal Services (Health/Beauty, Dry Cleaning, etc.)": "Non-Residential",
    "Wastewater Treatment Plant": "Non-Residential",
    "Municipal": "Non-Residential",
    "Mailing Center/Post Office": "Non-Residential",
}

eui_philadelphia_df["Property Type Category"] = eui_philadelphia_df["primary_prop_type_epa_calc"].map(property_type_classification)
eui_philadelphia_df.groupby("Property Type Category").size()

# we are goint to delete Property Type Category = Nan and Property Type Category = Other (136 obs)
eui_philadelphia_df = eui_philadelphia_df.dropna(subset=["primary_prop_type_epa_calc"])  
eui_philadelphia_df = eui_philadelphia_df[~eui_philadelphia_df["primary_prop_type_epa_calc"].isin(["Other"])] 


In [3]:
#1 kBtu = 0.2931 kWh
#1 ft² = 0.092903 m²

# Conversion factors

kBtu_to_kWh = 0.2931
ft2_to_m2 = 0.092903

eui_philadelphia_df = eui_philadelphia_df[eui_philadelphia_df["weather_norm_site_eui_kbtuft2"] > 0]

eui_philadelphia_df['EUI (kWh/m2/year)'] = (
    eui_philadelphia_df['weather_norm_site_eui_kbtuft2'] * kBtu_to_kWh / ft2_to_m2
)

eui_philadelphia_df["Residential EUI (kWh/m2/year)"] = eui_philadelphia_df.apply(
    lambda x: x["EUI (kWh/m2/year)"] if x["Property Type Category"] == "Residential" else None, axis=1
)

eui_philadelphia_df["Non-residential EUI (kWh/m2/year)"] = eui_philadelphia_df.apply(
    lambda x: x["EUI (kWh/m2/year)"] if x["Property Type Category"] == "Non-Residential" else None, axis=1
)

eui_philadelphia_df["Residential Area (ft²)"] = eui_philadelphia_df.apply(
    lambda x: x["total_floor_area_bld_pk_ft2"] if x["Property Type Category"] == "Residential" else None, axis=1
)

eui_philadelphia_df["Non-residential Area (ft²)"] = eui_philadelphia_df.apply(
    lambda x: x["total_floor_area_bld_pk_ft2"] if x["Property Type Category"] == "Non-Residential" else None, axis=1
)


In [4]:
eui_philadelphia_df = eui_philadelphia_df[[ 
    'latitude', 'longitude', 
    'Residential EUI (kWh/m2/year)', 'Non-residential EUI (kWh/m2/year)', 
    'Residential Area (ft²)', 'Non-residential Area (ft²)'
]]

eui_philadelphia_df.head()

,latitude,longitude,Residential EUI (kWh/m2/year),Non-residential EUI (kWh/m2/year),Residential Area (ft²),Non-residential Area (ft²)
0,39.95052011,-75.16619282,NaN,399.410783,NaN,62118.0
1,39.9512734,-75.16431263,NaN,343.253501,NaN,334050.0
2,39.94740488,-75.16624195,388.999602,NaN,95551.0,NaN
3,39.94883356,-75.16791508,126.196140,NaN,NaN,NaN
4,39.95680987,-75.16540056,NaN,534.125163,NaN,162086.0


In [5]:
eui_philadelphia_df.to_csv("../data/02_interim/eui_philadelphia.csv", index=False)